In [ ]:

!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7

!pip install -r requirements.txt

!curl -L "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip" -o coco128.zip
!unzip coco128.zip -d ./data/


yaml_content = """
train: data/coco128/images/train2017
val: data/coco128/images/train2017

nc: 80  # Number of classes in the COCO dataset
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
        'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush']
"""

with open('data/coco128.yaml', 'w') as f:
    f.write(yaml_content)


import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt -O yolov7.pt

import os
best_weights_path = 'runs/train/exp/weights/best.pt'

if os.path.exists(best_weights_path):
    print("Using trained weights for evaluation...")
    !python test.py --data data/coco128.yaml --img 640 --batch 16 --conf 0.001 --weights {best_weights_path} --device {device}
else:
    print("No trained weights found, using pre-trained yolov7.pt weights...")
    !python test.py --data data/coco128.yaml --img 640 --batch 16 --conf 0.001 --weights yolov7.pt --device {device}


if os.path.exists(best_weights_path):
    print("Using trained weights for inference...")
    !python detect.py --weights {best_weights_path} --conf 0.25 --source inference/images --device {device}
else:
    print("No trained weights found, using pre-trained yolov7.pt weights for inference...")
    !python detect.py --weights yolov7.pt --conf 0.25 --source inference/images --device {device}

import cv2
import matplotlib.pyplot as plt

image_path = 'runs/detect/exp/image0.jpg'
if os.path.exists(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
    else:
        print(f"Error: Image at path {image_path} could not be loaded.")
else:
    print(f"Error: {image_path} does not exist.")


fatal: destination path 'yolov7' already exists and is not an empty directory.
/content/yolov7
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6819k  100 6819k    0     0  7574k      0 --:--:-- --:--:-- --:--:-- 29.9M
Archive:  coco128.zip
replace ./data/coco128/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: Using device: cpu
--2024-09-16 09:43:54--  https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/511187726/b0243edf-9fb0-4337-95e1-42555f1b37cf?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20240

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Function to perform K-means clustering on the image
def kmeans_segmentation(image, k):
    # Reshape image into a 2D array of pixels (each pixel is represented by its RGB values)
    pixel_values = image.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)  # Convert to float for K-means

    # Define criteria and apply K-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(pixel_values)

    # Get the center of clusters (the new color for each cluster)
    centers = np.uint8(kmeans.cluster_centers_)

    # Map the label of each pixel to its corresponding cluster center
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image.shape)

    return segmented_image

# Function to display original and segmented images
def display_images(original, segmented):
    plt.figure(figsize=(10, 5))

    # Original image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')

    # Segmented image
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB))
    plt.title('Segmented Image')

    plt.show()

# Example usage (after image upload):
# image = cv2.imread('your_image.jpg')  # Replace with your uploaded image path
# segmented_img = kmeans_segmentation(image, k=3)
# display_images(image, segmented_img)
